Loading Functions

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests, time, re, math, openpyxl, datetime, os, shutil, psutil, platform, pyautogui, subprocess, webbrowser
from tqdm import *
import xlwings as xw
from selenium import webdriver

#from splinter import Browser
#from pandas.io.json import json_normalize

# Def get position function: 增加程序对数据源结构变化的稳定性
def f_p(df, feature):
    feature_list = df.iloc[:,0].get_values().tolist() # 第一列为特征名称，Index为连续自然数
    return [i for i,x in enumerate(feature_list) if x == feature][0]

# Def Check Nan Function
# https://stackoverflow.com/questions/944700/how-can-i-check-for-nan-in-python
def is_nan(x):
    return isinstance(x, float) and math.isnan(x)

# Get current file path for migration consistency
path = os.getcwd().replace('\\','/') #r'%s' % os.getcwd().replace('\\','/')

# 定义函数：从文本中根据数字类型提取Int或者Float数值
def find_number(text):
    if len(re.findall(r'\.',text)) == 0:
        return float(re.findall(r'\d+',text.replace(",", ""))[0])
    else:
        return float(re.findall(r'\d+\.\d+',text.replace(",", ""))[0])

# Excel App Kill
def kill_excel():
    try:
        for proc in psutil.process_iter():
            if 'EXCEL.EXE' in proc.name() or 'Microsoft Excel' in proc.name():
                proc.kill()
    except Exception as e:
        print (e)
            
# Chrome App Kill
def kill_chrome():
    try:
        for proc in psutil.process_iter():
            if 'chrome.exe' in proc.name():
                proc.kill()
    except Exception as e:
        print (e)

# Open WebPage in IE
def ie_open(url):
    ie = webbrowser.get(webbrowser.iexplore)
    ie.open(url)
    
# 定义ChromeDriver
# mac:把chromedriver放到/anaconda/bin
if platform.system() == 'Darwin':
    driver = webdriver.Chrome()
else:
    # Windows:把chromedriver放到下面的目录中
    chromedriver = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe"
    os.environ["webdriver.chrome.driver"] = chromedriver
    #driver = webdriver.Chrome(chromedriver)
    
# Download File Common Function
## 加入自主命名(根据列表)的功能
def download_file(url, path):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                #f.flush() commented by recommendation from J.F.Sebastian
    shutil.move(local_filename,path+'/'+local_filename)
    
# Convert TimeString to DateTime

# 定义天眼查爬虫
def scrap(keyword):

    def open_browser(url):
        driver = webdriver.Chrome()
        driver.get(url)
        return driver


    def log_in(driver):
        # 模拟登陆
        driver.find_element_by_xpath(
            ".//*[@id='web-content']/div/div/div/div[2]/div/div[2]/div[2]/div[2]/div[2]/input"). \
            send_keys(username)
        driver.find_element_by_xpath(
            ".//*[@id='web-content']/div/div/div/div[2]/div/div[2]/div[2]/div[2]/div[3]/input"). \
            send_keys(password)
        driver.find_element_by_xpath(
            ".//*[@id='web-content']/div/div/div/div[2]/div/div[2]/div[2]/div[2]/div[5]").click()
        time.sleep(3)
        return driver


    def search_company(driver, url1):
        driver.get(url1)
        content = driver.page_source.encode('utf-8')
        soup1 = BeautifulSoup(content, 'lxml')
        time.sleep(3)
        # company_name = soup1.find('a', class_="query_name sv-search-company f18 in-block vertical-middle").get_text()

        url2 = soup1.find('a', class_="query_name sv-search-company f18 in-block vertical-middle").attrs['href']

    #     ## 如果搜索有误，手工定义URL2地址
    #     url2 = 'https://www.tianyancha.com/company/28472888'

        driver.get(url2)
        return driver

    
    def get_base_info(driver):
        base_table = {}
#         base_table['名称'] = driver.find_element_by_xpath("//div[@class='company_header_width ie9Style position-rel']/div").text.split('我要认证')[0]
#         try:
#             base_info = driver.find_element_by_xpath("//div[@class='company_header_interior pl10 pt10 pb10 position-rel company-claim-header-bc mt15']")
#         except:
#             base_info = driver.find_element_by_xpath("//div[@class='company_header_interior pl10 pt10 pb10 position-rel company-claim-header-bc ']")
#         base_table['电话'] = base_info.text.split('电话：')[1].split('邮箱：')[0]
#         base_table['邮箱'] = base_info.text.split('邮箱：')[1].split('\n')[0]
#         base_table['网址'] = base_info.text.split('网址：')[1].split('地址')[0]
#         base_table['地址'] = base_info.text.split('地址：')[1].split('\n')[0]
#         try:
#             abstract = driver.find_element_by_xpath("//div[@class='sec-c2 over-hide']/script")
#             base_table['简介'] = driver.execute_script("return arguments[0].textContent", abstract).strip()
#         except:
#             abstract = driver.find_element_by_xpath("//div[@class='sec-c2 over-hide']")
#             base_table['简介'] = driver.execute_script("return arguments[0].textContent", abstract).strip()[3:]
#         tabs = driver.find_elements_by_tag_name('table')

#         rows1 = tabs[0].find_elements_by_tag_name('tr')
#         if len(rows1[1].find_elements_by_tag_name('td')[0].text.split('\n')[0]) < 2:
#             base_table['法人代表'] = rows1[1].find_elements_by_tag_name('td')[0].text.split('\n')[1]
#         else:
#             base_table['法人代表'] = rows1[1].find_elements_by_tag_name('td')[0].text.split('\n')[0]
#         base_table['注册资本'] = rows1[1].find_elements_by_tag_name('td')[1].text.split('\n')[1]
#         #base_table['注册时间'] = rows1[1].find_elements_by_tag_name('td')[1].text.split('\n')[3]
#         base_table['公司状态'] = rows1[1].find_elements_by_tag_name('td')[1].text.split('\n')[5]

#         rows2 = tabs[1].find_elements_by_tag_name('tr')
#         base_table['工商注册号'] = rows2[0].find_elements_by_tag_name('td')[1].text
#         base_table['统一信用代码'] = rows2[1].find_elements_by_tag_name('td')[1].text
#         base_table['纳税人识别号'] = rows2[2].find_elements_by_tag_name('td')[1].text
#         base_table['营业期限'] = rows2[3].find_elements_by_tag_name('td')[1].text
#         base_table['登记机关'] = rows2[4].find_elements_by_tag_name('td')[1].text

#         base_table['组织机构代码'] = rows2[0].find_elements_by_tag_name('td')[3].text
#         base_table['公司类型'] = rows2[1].find_elements_by_tag_name('td')[3].text
#         base_table['行业'] = rows2[2].find_elements_by_tag_name('td')[3].text
#         #base_table['核准日期'] = rows2[3].find_elements_by_tag_name('td')[3].text
#         base_table['英文名称'] = rows2[4].find_elements_by_tag_name('td')[3].text

#         base_table['注册地址'] = rows2[5].find_elements_by_tag_name('td')[1].text.split('附近公司')[0]
#         base_table['经营范围'] = rows2[6].find_elements_by_tag_name('td')[1].text

    #     # 信息修正
    #     base_table['注册时间'] = base_table['经营范围'].split['至'][0]
    #     base_table['核准日期'] = base_table['经营范围'].split['至'][0]
        print ('请手工更新注册时间/核准日期/注册资本！')

        return pd.DataFrame([base_table])

    
    # 特殊处理：主要人员
    ## staff_info定位不准？
    def get_staff_info(driver):
        staff_list = []
        staff_info = driver.find_elements_by_xpath("//div[@class='in-block f14 new-c5 pt9 pl10 overflow-width vertival-middle new-border-right']")
        for i in range(len(staff_info)):
            position = driver.find_elements_by_xpath("//div[@class='in-block f14 new-c5 pt9 pl10 overflow-width vertival-middle new-border-right']")[i].text
            person = driver.find_elements_by_xpath("//a[@class='overflow-width in-block vertival-middle pl15 mb4']")[i].text
            staff_list.append({'职位': position, '人员名称': person})
        staff_table = pd.DataFrame(staff_list, columns=['职位', '人员名称'])
        return staff_table

    
    # 特殊处理:上市公告
    ## 加入类别搜索功能
    def get_announcement_info(driver):
        announcement_df = pd.DataFrame(columns=['序号','日期','上市公告','上市公告网页链接']) ## 子函数自动获取columns
        ### 函数化
        content = driver.page_source.encode('utf-8')
        ## 能不能只Encode局部的driver
        soup = BeautifulSoup(content, 'lxml')
        announcement_info = soup.find('div',id='_container_announcement').find('tbody').find_all('tr')
        for i in range(len(announcement_info)):
            index = announcement_info[i].find_all('td')[0].get_text()
            date = announcement_info[i].find_all('td')[1].get_text()
            announcement = announcement_info[i].find_all('td')[2].get_text()
            announcement_url = 'https://www.tianyancha.com' + announcement_info[i].find_all('td')[2].find('a')['href']
            announcement_df = announcement_df.append({'序号':index,'日期':date,'上市公告':announcement,'上市公告网页链接':announcement_url}, ignore_index=True)
        ###

        ### 判断此表格是否有翻页功能:重新封装change_page函数
        announcement_table = driver.find_element_by_xpath("//div[contains(@id,'_container_announcement')]")
        onclickflag = tryonclick(announcement_table)
        if onclickflag == 1:
            PageCount = announcement_table.find_element_by_class_name('total').text
            PageCount = re.sub("\D", "", PageCount)  # 使用正则表达式取字符串中的数字 ；\D表示非数字的意思
            for i in range(int(PageCount) - 1):
                button = announcement_table.find_element_by_xpath(".//li[@class='pagination-next  ']/a")
                driver.execute_script("arguments[0].click();", button)
                ####################################################################################
                time.sleep(change_page_interval)
                ####################################################################################
        ###
                ### 函数化
                content = driver.page_source.encode('utf-8')
                ## 能不能只Encode局部的driver
                soup = BeautifulSoup(content, 'lxml')
                announcement_info = soup.find('div',id='_container_announcement').find('tbody').find_all('tr')
                for i in range(len(announcement_info)):
                    index = announcement_info[i].find_all('td')[0].get_text()
                    date = announcement_info[i].find_all('td')[1].get_text()
                    announcement = announcement_info[i].find_all('td')[2].get_text()
                    announcement_url = 'https://www.tianyancha.com' + announcement_info[i].find_all('td')[2].find('a')['href']
                    announcement_df = announcement_df.append({'序号':index,'日期':date,'上市公告':announcement,'上市公告网页链接':announcement_url}, ignore_index=True)
                ###
        return announcement_df

    
    def tryonclick(table): # table实质上是selenium WebElement
        # 测试是否有翻页
        ## 把条件判断写进tryoneclick中
        try:
            # 找到有翻页标记
            table.find_element_by_tag_name('ul')
            onclickflag = 1
        except Exception:
            print("没有翻页") ## 声明表格名称: name[x] + 
            onclickflag = 0
        return onclickflag

    def tryontap(table):
        # 测试是否有翻页
        try:
            table.find_element_by_xpath("//div[contains(@class,'over-hide changeTabLine f14')]")
            ontapflag = 1
        except Exception:
            print("没有时间切换页") ## 声明表格名称: name[x] + 
            ontapflag = 0
        return ontapflag

    def change_page(table, df):
        PageCount = table.find_element_by_class_name('total').text
        PageCount = re.sub("\D", "", PageCount)  # 使用正则表达式取字符串中的数字 ；\D表示非数字的意思
        for i in range(int(PageCount) - 1):
            button = table.find_element_by_xpath(".//li[@class='pagination-next  ']/a")
            driver.execute_script("arguments[0].click();", button)
            ####################################################################################
            time.sleep(change_page_interval) # 更新换页时间间隔,以应对反爬虫
            ####################################################################################
            df2 = get_table_info(table) ## 应该可以更换不同的get_XXXX_info
            df = df.append(df2)
        return df
    
    def change_tap(table, df):
        TapCount = len(table.find_elements_by_tag_name('div'))
        for i in range(int(TapCount) - 3):
            button = table.find_elements_by_tag_name('div')[i+3]
            driver.execute_script("arguments[0].click();", button)
            time.sleep(2)
            df2 = get_table_info(table) ## 应该可以更换不同的get_XXXX_info
     ##     df2['日期'] = table.find_elements_by_tag_name('div')[i+3].text
            df = df.append(df2, ignore_index=True)
     ## df = df.drop(columns=['序号'])
        return df
    
    def get_table_info(table):
        tab = table.find_element_by_tag_name('table')
        df = pd.read_html('<table>' + tab.get_attribute('innerHTML') + '</table>')
        if isinstance(df, list):
            df = df[0]
        if '操作' in df.columns:
            df = df.drop(columns='操作')
        return df

    
    def scrapy(driver):
        # Waiting time for volatilityNum to load
        time.sleep(2)
        
        tables = driver.find_elements_by_xpath("//div[contains(@id,'_container_')]")

        # 获取每个表格的名字
        c = '_container_'
        name = [0] * (len(tables) - 2)
        # 生成一个独一无二的十六位参数作为公司标记，一个公司对应一个，需要插入多个数据表
        id = keyword
        table_dict = {}
        for x in range(len(tables)-2):
            name[x] = tables[x].get_attribute('id')
            name[x] = name[x].replace(c, '')  # 可以用这个名称去匹配数据库
            # 判断是表格还是表单
            num = tables[x].find_elements_by_tag_name('table')

            # 基本信息表：table有两个
            if len(num) > 1: ##需要更好地设置baseinfo的判定条件
                table_dict[name[x]] = get_base_info(driver)
            
            #########
            # 排除列表：不同业务可以设置不同分类，实现信息的精准爬取
            #########
            elif name[x] in ['recruit', 'tmInfo', 'holdingCompany', 'bonus', 'invest', 'firmProduct', 'jingpin', \
                             'bid', 'taxcredit', 'certificate', 'patent', 'copyright', 'product', 'importAndExport', \
                             'copyrightWorks', 'wechat', 'websiteRecords', 'announcementcourt', 'lawsuit', 'court', \
                             'branch', 'touzi', 'judicialSale', 'bond', 'teamMember']:
                pass

#             # 公司高管的特殊处理
#             elif name[x] == 'staff':
#                 table_dict[name[x]] = get_staff_info(driver)
            
            # 公告的特殊处理：加入URL
            elif name[x] == 'announcement':
                table_dict[name[x]] = get_announcement_info(driver)                

            #  单纯的表格进行信息爬取
            ## 含头像的行未对齐
            elif len(num) == 1:
                df = get_table_info(tables[x])
                onclickflag = tryonclick(tables[x])
                ontapflag = tryontap(tables[x])
                # 判断此表格是否有翻页功能
                if onclickflag == 1:
                    df = change_page(tables[x], df)
            ##  if ontapflag == 1:
            ##      df = change_tap(tables[x], df)
                table_dict[name[x]] = df
                
            else:
                table_dict[name[x]] = pd.DataFrame()

        #table_dict['websiteRecords'] = get_table_info(tables[len(tables)-2])
        return table_dict


    def gen_excel(table_dict, keyword):
        with pd.ExcelWriter(path+'/'+keyword+'.xlsx') as writer:
            for sheet_name in table_dict:
                table_dict[sheet_name].to_excel(writer, sheet_name=sheet_name, index=None)
    
    # 微信通知提醒进度：已进行到'131. XXXX; 35%',声明序号/名称/完成度;发送给自己
    def notification_wechat():
        pass

    if __name__ == "__main__":
        url = 'https://www.tianyancha.com/login'
        url1 = 'http://www.tianyancha.com/search?key=%s&checkFrom=searchBox' % keyword

        driver = open_browser(url)
        driver = log_in(driver)
        driver = search_company(driver, url1)
        table_dict = scrapy(driver)
        gen_excel(table_dict, keyword)
        notification_wechat()

        # 个性操作
        os.makedirs(path+'/'+'clients'+'/'+ str(i+1) + '. ' + keyword + ' ' + keyword_list_name[i])
        shutil.move(path+'/'+keyword+'.xlsx',path+'/'+'clients'+'/'+ str(i+1) + '. ' + keyword + ' ' + keyword_list_name[i] +'/'+keyword + ' ' + keyword_list_name[i] + '.xlsx')

Input

In [ ]:
username = 'XXX' # 输入：天眼查注册名（手机号）
password = 'XXX' # 输入：天眼查注册密码
keyword = 'XXXX' # 输入: 查询公司的关键字

Execution

In [ ]:
kill_chrome()

try:
    scrap(keyword)
except Exception as e:
    print (e)
    break
    
kill_chrome()